## Single Customer View
Steps involved in creating SCV
### 1. Build base UserSCV table  
> 1.1. Cleanse the data (Validate Email, format Phone No, Landline No). This is done by calling function on each row in dataFrame. 

> 1.2. An intermediate table is created to hold validated/cleanse data, before transforming original data. 

> 1.3. Create additional fields by combining base fields (FirstName, UserName, LastName, DOB). Some of the combinations are as follows:
            * Firstname_Lastname_RegIP		
            * Firstname_Lastname_LastIP		
            * Firstname_Lastname_Username		
            * Firstname_DOB_City				
            * Firstname_Postcode				
            * Firstname_Mobilephone			
            * DOB_Postcode					
            * Address1_Postcode				
            * Firstname_Lastname_Address1_City
> Create UserSCV hive table with base fields and additional fields.


### 2. For each data load, perform check against base UserCSV. A record is considered same if it meets any one of the criteria:
| FirstName| Lastname | DOB  | Email | Postcode | Result   |
| :-------:| :-------:| :---:| :----:| :-------:| :-------:|
| X|X|X|X|X|**MATCH**|
|  |X|X|X|X|**MATCH**|
| X| |X|X|X|**MATCH**|
| X|X| |X|X|**MATCH**|
| X|X|X| |X|**MATCH**|
| X|X|X|X| |**MATCH**|

**Minimal conditions for match: **

| S.No| Criteria|
| :--:| :------:|
|1.|Firstname + IP Address|
|2.|Firstname + Username|

### 2.1. Data is given as  csv file and converted into Table with cleansed data. Join is performed with UserSCV table and loaded data and eac criteria mentioned above is checked to determine the match with existing Master UserSCV table.  
### 3. If matched records found, insert new version of user record with Related Id into UserSCV table. 


####  Issues faced while building UserSCV

1. **Pre-processing data**: Pre-processing and cleansing posed as main milestone when building base UserSCV table. A function is called on each row to pre-process the data.
2. **Transforming data:** Transforming pre-process data before converting into UserSCV table involves adding many new fields by combining different combinations of exisitin field and assigning each row with unique Id. This unique Id will be used as "Related Id" when matching user record is found in UserSCV table.

In [0]:
# import libraries
from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
import requests
import json
import phonenumbers 
import re
import datetime
import pandas as pd
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import  col
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
# schema for SCV User Table 
user_schema = StructType([
            StructField("id", IntegerType(), False),
            StructField("Userid", IntegerType(), True),
            StructField("SkinID", StringType(), True),
            StructField("username", StringType(), True),
            StructField("first_name", StringType(), True),
            StructField("last_name", StringType(), True),
            StructField("email", StringType(), True),
            StructField("gender", StringType(), True), 
            StructField("ip_address", StringType(), True), 
            StructField("RegDate", StringType(), True), 
            StructField("RegIP", StringType(), True), 
            StructField("LastIP", StringType(), True), 
            StructField("DOB", StringType(), True), 
            StructField("Postcode", StringType(), True), 
            StructField("MobilePhone", StringType(), True), 
            StructField("Landline", StringType(), True), 
            StructField("Address1", StringType(), True),
            StructField("City", StringType(), True),
            StructField("County", StringType(), True),
            StructField("Country", StringType(), True),
            StructField("SelfExcludedUntil", StringType(), True),
            StructField("Status", StringType(), True)])
            

In [0]:
def fixUserRow(c):
    # get the Mobile field
    number = c.MobilePhone

    # initialize variables 
    is_valid_number = "N"
    clean_number = None
    number_type = None
    valid_mail = None

    p = None

    if number is not None:
        # Clean the Mobile Number first
        try:
            p = phonenumbers.parse(number, c.Country)

            if phonenumbers.is_valid_number(p):
                is_valid_number = "Y"
            elif phonenumbers.truncate_too_long_number(p):
                is_valid_number = "Y"
            else:
                is_valid_number = "N"

            clean_number = "%s%s" % (p.country_code, p.national_number)
            
        except:
            p = None

    # clean up PhoneNumber
    phone_no = c.Landline
    if phone_no is not None:
      phone_no = phone_no.replace('-', '')
      if (len(phone_no) != 10):
        phone_no = None
    
    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.email):
      valid_mail = c.email
    
    return Row( 
		id = c.id, 
        Userid = c.Userid, 			
        SkinID = c.SkinID,
        username = c.username,
        first_name = c.first_name, 		
        last_name = c.last_name,	
        email = valid_mail,			
        gender = c.gender,			
        ip_address = c.ip_address,
        RegDate = c.RegDate,
        RegIP = c.RegIP,
		LastIP = c.LastIP,			
		DOB = c.DOB,			
		Postcode = c.Postcode,		
		MobilePhone = clean_number, 	
		Landline = phone_no, 		
		Address1 = c.Address1,		
        City = c.City, 			
		County = c.County,			
		Country = c.Country, 		
        SelfExcludedUntil = c.SelfExcludedUntil,
		Status = c.Status			
    )


In [0]:
# Read the csv file as example
csvFilePath = '/FileStore/tables/users_load.csv'
df = sqlContext.read.format('com.databricks.spark.csv') \
              .options(header=True).schema( user_schema).load(csvFilePath)

In [0]:
df.show()

+---+------+----------+--------------+----------+------------+--------------------+------+---------------+----------+---------------+---------------+----------+--------+-------------+------------+--------------------+-------------+------+-------+-----------------+------+
 id|Userid| SkinID| username|first_name| last_name| email|gender| ip_address| RegDate| RegIP| LastIP| DOB|Postcode| MobilePhone| Landline| Address1| City|County|Country|SelfExcludedUntil|Status|
+---+------+----------+--------------+----------+------------+--------------------+------+---------------+----------+---------------+---------------+----------+--------+-------------+------------+--------------------+-------------+------+-------+-----------------+------+
 1| 1| Violet| dchapier0| Daron| Chapier|dchapier0@busines...| Male| 181.38.232.147|06/13/2017| 117.141.184.55| 120.11.125.235|06/28/1975| 14276|(716) 4201216|518-545-3401| 4 Summit Lane| Buffalo| NY| US| 1/4/2018| true|
 2| 2| Orange| emourant1| Ethelind| Mourant|emourant1@science...|Female| 10.36.197.139|11/19/2017| 6.13.51.172| 132.29.114.56|01/18/1985| 11210|(347) 5401904|845-698-8568| 58 Dorton Plaza| Brooklyn| NY| US| 10/13/2017| false|
 3| 3| Indigo| rpenna2| Rex| Penna| rpenna2@topsy.com| Male| 122.34.37.81|02/07/2018| 115.172.250.73| 118.97.234.122|07/29/1999| 10024|(646) 4422808|212-929-5500|2139 Cordelia Ter...|New York City| NY| US| 6/9/2017| true|
 4| 4| Mauv| cleaning3| Consalve| Leaning| cleaning3@1688.com| Male| 235.5.137.189|11/23/2017| 44.91.93.110| 141.40.183.98|03/02/1991| 10125|(212) 9259468|716-461-4206| 0327 Erie Pass|New York City| NY| US| 10/17/2017| true|
 5| 5| Indigo| lpocknell4| Letta| Pocknell| lpocknell4@vk.com|Female| 229.167.156.39|04/08/2018|175.178.102.137| 108.159.183.58|12/20/1987| 11205|(718) 1309292|212-175-8529| 771 Eliot Plaza| Brooklyn| NY| US| 11/28/2017| true|
 6| 6| Mauv| gmacfaell5|Gilbertine| MacFaell|gmacfaell5@senate...|Female| 208.248.14.221|11/17/2017|145.207.102.182| 202.60.1.159|09/24/1995| 11470|(917) 5368502|518-745-1754|436 Lakewood Gard...| Jamaica| NY| US| 6/5/2017| true|
 7| 7| Pink| rivoshin6| Reba| Ivoshin|rivoshin6@newsvin...|Female| 25.198.179.229|08/20/2017| 226.79.95.197| 2.248.111.106|06/29/1997| 10310|(914) 3665367|212-451-0375|12785 Ludington P...|Staten Island| NY| US| 9/10/2017| false|
 8| 8|Aquamarine| dpharro7| Devlin| Pharro| dpharro7@jugem.jp| Male|196.236.216.165|07/23/2017| 11.24.158.185| 81.145.159.208|08/28/1970| 10115|(212) 8875603|347-966-6081| 189 Nova Drive|New York City| NY| US| 12/26/2017| false|
 9| 9| Crimson|jchomiszewski8| Julietta|Chomiszewski|jchomiszewski8@la...|Female| 176.202.20.181|05/22/2017| 39.236.247.143| 21.144.181.1|01/03/1986| 14215|(716) 5918889|716-403-7731|5483 Burning Wood...| Buffalo| NY| US| 1/13/2018| true|
 10| 10| Goldenrod| adepport9| Antony| Depport|adepport9@blogtal...| Male| 38.190.195.188|03/30/2018| 99.176.18.199| 228.239.14.15|07/31/1972| 11480|(917) 5423379|917-346-3101| 3 Jenifer Circle| Jamaica| NY| US| 2/8/2018| true|
 11| 11| Turquoise| fwalasa| Flo| Walas|fwalasa@cbslocal.com|Female| 26.143.222.207|03/24/2018| 40.15.221.126|189.159.234.205|12/12/1981| 12232|(518) 2686993|646-528-2225| 71 Sunbrook Road| Albany| NY| US| 1/12/2018| false|
 12| 12| Pink| lsiveterb| Latashia| Siveter|lsiveterb@privacy...|Female| 71.160.191.131|09/13/2017| 32.152.51.122| 78.165.145.214|09/15/1985| 10305|(718) 6412282|718-708-4351|27 Schlimgen Parkway|Staten Island| NY| US| 5/31/2017| false|
 13| 13| Maroon| ebuggc| Emilee| Bugg| ebuggc@g.co|Female| 114.227.84.113|11/29/2017|124.247.138.130|220.113.134.247|05/31/1995| 10203|(212) 2713202|212-698-0829| 91 Schmedeman Park|New York City| NY| US| 3/24/2018| true|
 14| 14| Blue| ehattd|Enrichetta| Hatt| ehattd@meetup.com|Female| 254.197.42.247|12/23/2017| 78.50.124.211| 196.142.50.219|09/03/1973| 10009|(646) 4232270|516-386-5123| 47981 Russell Court|New York City| NY| US| 4/11/2018| false|
 15| 15| Fuscia| dsibthorpe| Dallas| Sibthorp|dsibthorpe@pcworl...| Mal

In [0]:
# cleanse the data
# Standardise the telephone number to take away -, ) from input data 

df_user_updated = df.rdd.map(lambda c: fixUserRow(c))
# change the column type now
df_user = sqlContext.createDataFrame(df_user_updated, user_schema)
 

In [0]:
df_user.show(5)

+---+------+------+----------+----------+---------+--------------------+------+--------------+----------+---------------+--------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+
 id|Userid|SkinID| username|first_name|last_name| email|gender| ip_address| RegDate| RegIP| LastIP| DOB|Postcode|MobilePhone| Landline| Address1| City|County|Country|SelfExcludedUntil|Status|
+---+------+------+----------+----------+---------+--------------------+------+--------------+----------+---------------+--------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+
 1| 1|Violet| dchapier0| Daron| Chapier|dchapier0@busines...| Male|181.38.232.147|06/13/2017| 117.141.184.55|120.11.125.235|06/28/1975| 14276|17164201216|5185453401| 4 Summit Lane| Buffalo| NY| US| 1/4/2018| true|
 2| 2|Orange| emourant1| Ethelind| Mourant|emourant1@science...|Female| 10.36.197.139|11/19/2017| 6.13.51.172| 132.29.114.56|01/18/1985| 11210|13475401904|8456988568| 58 Dorton Plaza| Brooklyn| NY| US| 10/13/2017| false|
 3| 3|Indigo| rpenna2| Rex| Penna| rpenna2@topsy.com| Male| 122.34.37.81|02/07/2018| 115.172.250.73|118.97.234.122|07/29/1999| 10024|16464422808|2129295500|2139 Cordelia Ter...|New York City| NY| US| 6/9/2017| true|
 4| 4| Mauv| cleaning3| Consalve| Leaning| cleaning3@1688.com| Male| 235.5.137.189|11/23/2017| 44.91.93.110| 141.40.183.98|03/02/1991| 10125|12129259468|7164614206| 0327 Erie Pass|New York City| NY| US| 10/17/2017| true|
 5| 5|Indigo|lpocknell4| Letta| Pocknell| lpocknell4@vk.com|Female|229.167.156.39|04/08/2018|175.178.102.137|108.159.183.58|12/20/1987| 11205|17181309292|2121758529| 771 Eliot Plaza| Brooklyn| NY| US| 11/28/2017| true|
+---+------+------+----------+----------+---------+--------------------+------+--------------+----------+---------------+--------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+
only showing top 5 rows

In [0]:
spark.sql('DROP TABLE IF EXISTS users_load')        


# write df to db
# df.write.jdbc(jdbc_url, 'users_load', mode='overwrite', properties=connectionProperties)
df_user.write.saveAsTable("users_load")


In [0]:
# create output table
df = df_user
dateTimeStr = datetime.datetime.today().strftime("%m-%d-%Y %H:%M:%S")


userSCV =  df.withColumn("ID", F.monotonically_increasing_id()) \
.withColumn("RelatedID", lit(-1).cast(IntegerType())) 
userSCV = userSCV.withColumn("EntityId", col("ID")) 

# rename columns 
userSCV = userSCV.withColumn("OriginalEmail", col("email")) 
userSCV = userSCV.withColumn("OriginalFirstname", col("first_name")) 
userSCV = userSCV.withColumn("OriginalLastname", col("last_name")) 
userSCV = userSCV.withColumn("OriginalRegDate", col("RegDate"))
userSCV = userSCV.withColumn("OriginalDOB", col("DOB"))
userSCV = userSCV.withColumn("OriginalPostcode", col("Postcode"))             
userSCV = userSCV.withColumn("OriginalMobilePhone", col("MobilePhone"))
userSCV = userSCV.withColumn("OriginalAddress1", col("Address1"))            
#userSCV = userSCV.withColumn("OriginalAddress2", col("Address2"))            
userSCV = userSCV.withColumn("OriginalCity", col("City"))
userSCV = userSCV.withColumn("Firstname_Lastname_RegIP", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('RegIP') ))       
userSCV = userSCV.withColumn("Firstname_Lastname_LastIP", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('LastIP') ))
userSCV = userSCV.withColumn("Firstname_Lastname_Username", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('Username') ))
userSCV = userSCV.withColumn("Firstname_DOB_City", F.concat(col('first_name'),lit('_'), col('DOB'), lit('_'),col('City') ))
userSCV = userSCV.withColumn("Firstname_Postcode", F.concat(col('first_name'),lit('_'), col('Postcode')  )) 
userSCV = userSCV.withColumn("Firstname_Mobilephone", F.concat(col('first_name'),lit('_'), col('MobilePhone')  ))          
userSCV = userSCV.withColumn("DOB_Postcode", F.concat(col('DOB'),lit('_'), col('Postcode')  )) 
userSCV = userSCV.withColumn("Address1_Postcode", F.concat(col('Address1'),lit('_'), col('Postcode')  ))              
userSCV = userSCV.withColumn("Firstname_Lastname_Address1_City", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('Address1'), lit('_'), col('City') ))
userSCV = userSCV.withColumn("Load_date", lit(dateTimeStr))
userSCV = userSCV.withColumn("LastModifiedDate", lit(dateTimeStr))
userSCV = userSCV.withColumn("CompareStatus", lit(0))
userSCV = userSCV.withColumn("CompareStatus", lit(None).cast(StringType()))

In [0]:
userSCV.show(5)

+---+------+------+----------+----------+---------+--------------------+------+--------------+----------+---------------+--------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+---------+--------+--------------------+-----------------+----------------+---------------+-----------+----------------+-------------------+--------------------+-------------+------------------------+-------------------------+---------------------------+--------------------+------------------+---------------------+----------------+--------------------+--------------------------------+-------------------+-------------------+-------------+
 ID|Userid|SkinID| username|first_name|last_name| email|gender| ip_address| RegDate| RegIP| LastIP| DOB|Postcode|MobilePhone| Landline| Address1| City|County|Country|SelfExcludedUntil|Status|RelatedID|EntityId| OriginalEmail|OriginalFirstname|OriginalLastname|OriginalRegDate|OriginalDOB|OriginalPostcode|OriginalMobilePhone| OriginalAddress1| OriginalCity|Firstname_Lastname_RegIP|Firstname_Lastname_LastIP|Firstname_Lastname_Username| Firstname_DOB_City|Firstname_Postcode|Firstname_Mobilephone| DOB_Postcode| Address1_Postcode|Firstname_Lastname_Address1_City| Load_date| LastModifiedDate|CompareStatus|
+---+------+------+----------+----------+---------+--------------------+------+--------------+----------+---------------+--------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+---------+--------+--------------------+-----------------+----------------+---------------+-----------+----------------+-------------------+--------------------+-------------+------------------------+-------------------------+---------------------------+--------------------+------------------+---------------------+----------------+--------------------+--------------------------------+-------------------+-------------------+-------------+
 0| 1|Violet| dchapier0| Daron| Chapier|dchapier0@busines...| Male|181.38.232.147|06/13/2017| 117.141.184.55|120.11.125.235|06/28/1975| 14276|17164201216|5185453401| 4 Summit Lane| Buffalo| NY| US| 1/4/2018| true| -1| 0|dchapier0@busines...| Daron| Chapier| 06/13/2017| 06/28/1975| 14276| 17164201216| 4 Summit Lane| Buffalo| Daron_Chapier_117...| Daron_Chapier_120...| Daron_Chapier_dch...|Daron_06/28/1975_...| Daron_14276| Daron_17164201216|06/28/1975_14276| 4 Summit Lane_14276| Daron_Chapier_4 S...|06-04-2018 13:28:18|06-04-2018 13:28:18| null|
 1| 2|Orange| emourant1| Ethelind| Mourant|emourant1@science...|Female| 10.36.197.139|11/19/2017| 6.13.51.172| 132.29.114.56|01/18/1985| 11210|13475401904|8456988568| 58 Dorton Plaza| Brooklyn| NY| US| 10/13/2017| false| -1| 1|emourant1@science...| Ethelind| Mourant| 11/19/2017| 01/18/1985| 11210| 13475401904| 58 Dorton Plaza| Brooklyn| Ethelind_Mourant_...| Ethelind_Mourant_...| Ethelind_Mourant_...|Ethelind_01/18/19...| Ethelind_11210| Ethelind_13475401904|01/18/1985_11210|58 Dorton Plaza_1...| Ethelind_Mourant_...|06-04-2018 13:28:18|06-04-2018 13:28:18| null|
 2| 3|Indigo| rpenna2| Rex| Penna| rpenna2@topsy.com| Male| 122.34.37.81|02/07/2018| 115.172.250.73|118.97.234.122|07/29/1999| 10024|16464422808|2129295500|2139 Cordelia Ter...|New York City| NY| US| 6/9/2017| true| -1| 2| rpenna2@topsy.com| Rex| Penna| 02/07/2018| 07/29/1999| 10024| 16464422808|2139 Cordelia Ter...|New York City| Rex_Penna_115.172...| Rex_Penna_118.97....| Rex_Penna_rpenna2|Rex_07/29/1999_Ne...| Rex_10024| Rex_16464422808|07/29/1999_10024|2139 Cordelia Ter...| Rex_Penna_2139 Co...|06-04-2018 13:28:18|06-04-2018 13:28:18| null|
 3| 4| Mauv| cleaning3| Consalve| Leaning| cleaning3@1688.com| Male| 235.5.137.189|11/23/2017| 44.91.93.110| 141.40.183.98|03/02/1991| 10125|12129259468|7164614206| 0327 Erie Pass|New York City| NY| US| 10/17/2017| true| -1| 3| cleaning3@1688.com| Consalve| Leaning| 11/23/2017| 03/02/1991| 10125| 12129259468| 0327 Erie Pass|New

In [0]:
# Create output Table
spark.sql('DROP TABLE IF EXISTS UserSCV')        

# Create a HIVE table to save Data fro Dataframe 
userSCV.write.saveAsTable("UserSCV")


In [0]:
def getDataFrameFromCSV(csvFileLocation, schema):
   
  df_new_load = sqlContext.read.format('com.databricks.spark.csv') \
                          .options(header=True).schema( schema).load(csvFileLocation)
  from pyspark.sql.functions import col

  
  # cleanse the data
  df_user_updated1 = df_new_load.rdd.map(lambda c: fixUserRow(c))
  # change the column type now
  df_new = sqlContext.createDataFrame(df_user_updated1, user_schema)
  df_new = df_new.select (col("Userid").alias("Userid1"), col("SkinID").alias("SkinID1"), \
                        col("username").alias("username1"), col("first_name").alias("first_name1"), \
                        col("last_name").alias("last_name1"), col("email").alias("email1"), \
                        col("gender").alias("gender1"), col("ip_address").alias("ip_address1"), \
                        col("RegDate").alias("RegDate1"), col("RegIP").alias("RegIP1"), \
                        col("LastIP").alias("LastIP1"), col("DOB").alias("DOB1"), \
                        col("Postcode").alias("Postcode1"), col("MobilePhone").alias("MobilePhone1"), \
                        col("Landline").alias("Landline1"), col("Address1").alias("Address11"), \
                        col("City").alias("City1"), col("County").alias("County1"), \
                        col("Country").alias("Country1"), col("SelfExcludedUntil").alias("SelfExcludedUntil1"), \
                        col("Status").alias("Status1")) 
  return df_new
  

In [0]:
# insert matching records into UserSCV table
def insertNewVersionOfUser(tableName):
  df = spark.sql("select  * from " + tableName)
  dateTimeStr = datetime.datetime.today().strftime("%m-%d-%Y %H:%M:%S")

  # select max of id from userSCV table
  lv = sqlContext.sql("select max(ID) as lastVal from UserSCV").collect()
  lastValue = lv[0]["lastVal"]
  df_userSCV = df.select("ID", \
                         "Userid1", \
                         "SkinID1", \
                         "username1", \
                         "first_name1", \
                         "last_name1", \
                         "email1", \
                         "gender1", "ip_address1", "RegDate1", "RegIP1", \
                         "LastIP1", "DOB1", "Postcode1", "MobilePhone1", "Landline1", \
                         "Address11", "City1", "County1", "Country1", \
                         "SelfExcludedUntil1", "Status1", \
                         "EntityId", \
                         "OriginalEmail", \
                         "OriginalFirstname", \
                         "OriginalLastname", \
                         "OriginalRegDate", \
                         "OriginalDOB", \
                         "OriginalPostcode", \
                         "OriginalMobilePhone", \
                         "OriginalAddress1", \
                         "OriginalCity", \
                         "Firstname_Lastname_RegIP", \
                         "Firstname_Lastname_LastIP", \
                         "Firstname_Lastname_Username", \
                         "Firstname_DOB_City",\
                         "Firstname_Postcode", \
                         "Firstname_Mobilephone", \
                         "DOB_Postcode",  \
                         "Address1_Postcode", \
                         "Firstname_Lastname_Address1_City")
  #df_userSCV = df_userSCV.withColumnRenamed("ID", "RelatedID") 
  df_userSCV = df_userSCV.withColumn("RelatedID", col("ID"))
  df_userSCV = df_userSCV.withColumn("Load_date", lit(dateTimeStr))
  df_userSCV = df_userSCV.withColumn("LastModifiedDate", lit(dateTimeStr))
  df_userSCV = df_userSCV.withColumn("CompareStatus", lit(0))
   
  #df_userSCV = df_userSCV.withColumn("ID", monotonically_increasing_id() + lastValue)
  df_userSCV = df_userSCV.select("ID", \
                        col("Userid1").alias("Userid"), col("SkinID1").alias("SkinID"), \
                        col("username1").alias("username"), col("first_name1").alias("first_name"), \
                        col("last_name1").alias("last_name"), col("email1").alias("email"), \
                        col("gender1").alias("gender"), col("ip_address1").alias("ip_address"), \
                        col("RegDate1").alias("RegDate"), col("RegIP1").alias("RegIP"), \
                        col("LastIP1").alias("LastIP"), col("DOB1").alias("DOB"), \
                        col("Postcode1").alias("Postcode"), col("MobilePhone1").alias("MobilePhone"), \
                        col("Landline1").alias("Landline"), col("Address11").alias("Address1"), \
                        col("City1").alias("City"), col("County1").alias("County"), \
                        col("Country1").alias("Country"), col("SelfExcludedUntil1").alias("SelfExcludedUntil"), \
                        col("Status1").alias("Status"), \
                         "RelatedID", \
                         "EntityId", \
                         "OriginalEmail", \
                         "OriginalFirstname", \
                         "OriginalLastname", \
                         "OriginalRegDate", \
                         "OriginalDOB", \
                         "OriginalPostcode", \
                         "OriginalMobilePhone", \
                         "OriginalAddress1", \
                         "OriginalCity", \
                         "Firstname_Lastname_RegIP", \
                         "Firstname_Lastname_LastIP", \
                         "Firstname_Lastname_Username", \
                         "Firstname_DOB_City",\
                         "Firstname_Postcode", \
                         "Firstname_Mobilephone", \
                         "DOB_Postcode",  \
                         "Address1_Postcode", \
                         "Firstname_Lastname_Address1_City", \
                         "Load_date", \
                         "LastModifiedDate",\
                         "CompareStatus")
  output_tbl_name = tableName + "_op" 
  df_userSCV.createOrReplaceTempView(output_tbl_name)
  
  
  if (tableName == "c1_FN_IP"):
    output_detailed_tbl_name = "c1_FirstName_IP_output"
  elif (tableName == "c1_FN_username"):
    output_detailed_tbl_name = "c1_FirstName_username_output"
  elif (tableName == "c1_fn_dob_city"):
    output_detailed_tbl_name = "c1_FirstName_dob_city_output"
  else:
    output_detailed_tbl_name = tableName + "_output"
  spark.sql('DROP TABLE IF EXISTS ' + output_detailed_tbl_name)
  sqlStr = "create table " + output_detailed_tbl_name + " as select * from " + output_tbl_name 
  sqlContext.sql(sqlStr);
  

  df_userSCV.write.insertInto("UserSCV")

In [0]:
# Read the csv file
# Format the row and change column names for joining 2 tables
# This is to check 1st criteria: FirstName + IP 
csvFilePath_new = '/FileStore/tables/user_criteria1.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)

In [0]:
# check for the minimal condition
# whether firstName + IP equals

df_criteria_min = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.ip_address == df_new.ip_address1) )
df_criteria_min.count()
df_criteria_min.createOrReplaceTempView("c1_FN_IP") 
spark.sql('DROP TABLE IF EXISTS c1_FirstName_IP_data')        
sqlContext.sql("create table c1_FirstName_IP_data as select * from c1_FN_IP");


In [0]:
df_criteria_min_count = df_criteria_min.count()
if (df_criteria_min_count > 0):
  insertNewVersionOfUser("c1_FN_IP")

In [0]:
# check for firstName + Username equals

# Read the csv file
# Format the row and change column names for joining 2 tables
# This is to check  criteria: FirstName + username 
csvFilePath_new = '/FileStore/tables/user_criteria2.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
df_criteria_fn_username = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.username == df_new.username1) )
df_criteria_fn_username.count()
df_criteria_fn_username.createOrReplaceTempView("c1_FN_username") 
spark.sql('DROP TABLE IF EXISTS c1_FirstName_username_data')        
sqlContext.sql("create table c1_FirstName_username_data as select * from c1_FN_username");


In [0]:
df_criteria_fn_username_count = df_criteria_fn_username.count()
if (df_criteria_fn_username_count > 0):
  insertNewVersionOfUser("c1_FN_username")

In [0]:
# check for firstName, Dob and city
# Read the csv file
# This is to check  criteria: FirstName + DOB + city 
csvFilePath_new = '/FileStore/tables/user_criteria3.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
df_criteria_fn_dob_city = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) \
                                       & (userSCV.DOB == df_new.DOB1) \
                                       & (userSCV.City == df_new.City1) )
df_criteria_fn_dob_city.count()
df_criteria_fn_dob_city.createOrReplaceTempView("c1_fn_dob_city") 
spark.sql('DROP TABLE IF EXISTS c1_FirstName_dob_city_data')        
sqlContext.sql("create table c1_FirstName_dob_city_data as select * from c1_fn_dob_city");

In [0]:
df_criteria_fn_dob_city_count = df_criteria_fn_dob_city.count()
if (df_criteria_fn_dob_city_count > 0):
  insertNewVersionOfUser("c1_fn_dob_city")

In [0]:
# check for firstName, postcode
# Read the csv file
# This is to check  criteria: FirstName + postcode 
csvFilePath_new = '/FileStore/tables/user_criteria3.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
df_criteria_fn_postcode = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) \
                                       & (userSCV.Postcode == df_new.Postcode1) )
df_criteria_fn_postcode.count()
df_criteria_fn_postcode.createOrReplaceTempView("c1_fn_postcode") 
spark.sql('DROP TABLE IF EXISTS c1_FirstName_postcode_data')        
sqlContext.sql("create table c1_FirstName_postcode_data as select * from c1_fn_dob_city");

In [0]:
df_criteria_fn_postcode_count = df_criteria_fn_postcode.count()
if (df_criteria_fn_postcode_count > 0):
  insertNewVersionOfUser("c1_fn_postcode")

In [0]:
# check for DOB, postcode
# Read the csv file
# This is to check  criteria: DOB + postcode 
csvFilePath_new = '/FileStore/tables/user_criteria3.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
df_criteria_postcode_dob = userSCV.join(df_new, (userSCV.DOB == df_new.DOB1) \
                                       & (userSCV.Postcode == df_new.Postcode1) )
df_criteria_postcode_dob.count()
df_criteria_postcode_dob.createOrReplaceTempView("c1_postcode_dob") 
spark.sql('DROP TABLE IF EXISTS c1_postcode_dob_data')        
sqlContext.sql("create table c1_postcode_dob_data as select * from c1_postcode_dob");


df_criteria_postcode_dob_count = df_criteria_postcode_dob.count()
if (df_criteria_fn_postcode_count > 0):
  insertNewVersionOfUser("c1_postcode_dob")

In [0]:
# check for aDDRESS1, postcode
# Read the csv file
# This is to check  criteria: Address1 + postcode 
csvFilePath_new = '/FileStore/tables/user_criteria3.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
df_criteria_postcode_addr1 = userSCV.join(df_new, (userSCV.Address1 == df_new.Address11) \
                                       & (userSCV.Postcode == df_new.Postcode1) )
df_criteria_postcode_addr1.count()
df_criteria_postcode_addr1.createOrReplaceTempView("c1_postcode_addr1") 
spark.sql('DROP TABLE IF EXISTS c1_postcode_addr1_data')        
sqlContext.sql("create table c1_postcode_addr1_data as select * from c1_postcode_addr1");


df_criteria_postcode_addr1_count = df_criteria_postcode_addr1.count()
if (df_criteria_postcode_addr1_count > 0):
  insertNewVersionOfUser("c1_postcode_addr1")

In [0]:
# check for firstName, lastName, Address1 and city
# Read the csv file
# This is to check  criteria: FirstName + LastName + Address1 + city 
csvFilePath_new = '/FileStore/tables/user_criteria4.csv'
df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
df_criteria_fn_ln_addr1_city = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) \
                                       & (userSCV.last_name == df_new.last_name1)
                                       & (userSCV.Address1 == df_new.Address11) \
                                       & (userSCV.City == df_new.City1) )
df_criteria_fn_ln_addr1_city.count()
df_criteria_fn_ln_addr1_city.createOrReplaceTempView("c1_fn_ln_addr1_city") 
spark.sql('DROP TABLE IF EXISTS c1_FirstName_lastname_addr1_data')        
sqlContext.sql("create table c1_FirstName_lastname_addr1_data as select * from c1_fn_ln_addr1_city");

In [0]:
df_criteria_fn_ln_addr1_city_count = df_criteria_fn_ln_addr1_city.count()
if (df_criteria_fn_ln_addr1_city_count > 0):
  insertNewVersionOfUser("c1_fn_ln_addr1_city")

In [0]:
# now we have base userSCV and next load data to be compared
# check for FirstName, LastName, DOB, Email and Postcode
df_criteria1 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.last_name == df_new.last_name1) \
                 & (userSCV.DOB == df_new.DOB1) & (userSCV.email == df_new.email1) \
                 & (userSCV.Postcode == df_new.Postcode1))


In [0]:
c1_count = df_criteria1.count()
print(c1_count)
if (c1_count > 0):
  df_criteria1.createOrReplaceTempView("criteria1")
  insertNewVersionOfUser("criteria1")

0

In [0]:
# check for LastName, DOB, Email and Postcode
df_criteria2 = userSCV.join(df_new,  (userSCV.last_name == df_new.last_name1) \
                 & (userSCV.DOB == df_new.DOB1) & (userSCV.email == df_new.email1) \
                 & (userSCV.Postcode == df_new.Postcode1))
df_criteria2.count()

Out[ 64 ]: 0

In [0]:
c2_count = df_criteria2.count()
if (c2_count > 0):
  df_criteria2.createOrReplaceTempView("criteria2")
  insertNewVersionOfUser("criteria2")

In [0]:
# check for FirstName, DOB, Email and Postcode
df_criteria3 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1)  \
                 & (userSCV.DOB == df_new.DOB1) & (userSCV.email == df_new.email1) \
                 & (userSCV.Postcode == df_new.Postcode1))


In [0]:
c3_count = df_criteria3.count()
if (c3_count > 0):
  df_criteria3.createOrReplaceTempView("criteria3")
  insertNewVersionOfUser("criteria3")

In [0]:
# check for FirstName, lastName, Email and Postcode
# check for FirstName, DOB, Email and Postcode
df_criteria4 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1)  \
                 & (userSCV.last_name == df_new.last_name1) & (userSCV.email == df_new.email1) \
                 & (userSCV.Postcode == df_new.Postcode1))

In [0]:
df_criteria4.printSchema()

root
-- ID: long (nullable = false)
-- Userid: integer (nullable = true)
-- SkinID: string (nullable = true)
-- username: string (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- email: string (nullable = true)
-- gender: string (nullable = true)
-- ip_address: string (nullable = true)
-- RegDate: string (nullable = true)
-- RegIP: string (nullable = true)
-- LastIP: string (nullable = true)
-- DOB: string (nullable = true)
-- Postcode: string (nullable = true)
-- MobilePhone: string (nullable = true)
-- Landline: string (nullable = true)
-- Address1: string (nullable = true)
-- City: string (nullable = true)
-- County: string (nullable = true)
-- Country: string (nullable = true)
-- SelfExcludedUntil: string (nullable = true)
-- Status: string (nullable = true)
-- RelatedID: integer (nullable = false)
-- EntityId: long (nullable = false)
-- OriginalEmail: string (nullable = true)
-- OriginalFirstname: string (nullable = true)
-- OriginalLastname: string (nullable = true)
-- OriginalRegDate: string (nullable = true)
-- OriginalDOB: string (nullable = true)
-- OriginalPostcode: string (nullable = true)
-- OriginalMobilePhone: string (nullable = true)
-- OriginalAddress1: string (nullable = true)
-- OriginalCity: string (nullable = true)
-- Firstname_Lastname_RegIP: string (nullable = true)
-- Firstname_Lastname_LastIP: string (nullable = true)
-- Firstname_Lastname_Username: string (nullable = true)
-- Firstname_DOB_City: string (nullable = true)
-- Firstname_Postcode: string (nullable = true)
-- Firstname_Mobilephone: string (nullable = true)
-- DOB_Postcode: string (nullable = true)
-- Address1_Postcode: string (nullable = true)
-- Firstname_Lastname_Address1_City: string (nullable = true)
-- Load_date: string (nullable = false)
-- LastModifiedDate: string (nullable = false)
-- CompareStatus: string (nullable = true)
-- Userid1: integer (nullable = true)
-- SkinID1: string (nullable = true)
-- username1: string (nullable = true)
-- first_name1: string (nullable = true)
-- last_name1: string (nullable = true)
-- email1: string (nullable = true)
-- gender1: string (nullable = true)
-- ip_address1: string (nullable = true)
-- RegDate1: string (nullable = true)
-- RegIP1: string (nullable = true)
-- LastIP1: string (nullable = true)
-- DOB1: string (nullable = true)
-- Postcode1: string (nullable = true)
-- MobilePhone1: string (nullable = true)
-- Landline1: string (nullable = true)
-- Address11: string (nullable = true)
-- City1: string (nullable = true)
-- County1: string (nullable = true)
-- Country1: string (nullable = true)
-- SelfExcludedUntil1: string (nullable = true)
-- Status1: string (nullable = true)

In [0]:
c4_count = df_criteria4.count()
if (c4_count > 0):
  df_criteria4.createOrReplaceTempView("criteria4")
  insertNewVersionOfUser("criteria4")

In [0]:
# check for FirstName, LastName, DOB and Postcode
df_criteria5 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.last_name == df_new.last_name1) \
                 & (userSCV.DOB == df_new.DOB1)  \
                 & (userSCV.Postcode == df_new.Postcode1))
df_criteria5.count()

Out[ 71 ]: 0

In [0]:
# check for FirstName, LastName, DOB, Email and Postcode
df_criteria6 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.last_name == df_new.last_name1) \
                 & (userSCV.DOB == df_new.DOB1) & (userSCV.email == df_new.email1) )
df_criteria6.count()


Out[ 72 ]: 0

In [0]:
# check for email alone
df_criteria_email = userSCV.join(df_new, (userSCV.email == df_new.email1))
df_criteria_email.count()

Out[ 73 ]: 1

In [0]:
email_count = df_criteria_email.count()
if (email_count > 0):
  df_criteria_email.createOrReplaceTempView("emailview")
  insertNewVersionOfUser("emailview")